In [ ]:
!pip install yahoofinancials
%pip install yahoofinancials

In [ ]:
import datetime
from typing import Tuple, List, Optional

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from yahoofinancials import YahooFinancials

In [ ]:
FIXED_SCENARIO_RETURN = .07

# Load data

In [ ]:
DATE_FORMAT = "%d.%m.%y"
DATE_COLUMN = "Datum"

In [ ]:
def get_avg_return_scenario(data: pd.DataFrame, avg_return: float, data_key: str = "Depotwert") -> \
        List[float]:
    days_list = [(data[DATE_COLUMN].iloc[i] - data[DATE_COLUMN].iloc[0]).days for i in
                 range(len(data[DATE_COLUMN]))]
    return [data[data_key].iloc[0] * (1 + avg_return) ** (
        (i / 365)) for i in days_list]

In [ ]:
def load_net_worth_history(filename: str) -> Tuple[pd.DataFrame, float]:
    """Load data from csv file and return it as a pandas dataframe"""
    df = pd.read_csv(filename, sep=";", decimal=".", parse_dates=[DATE_COLUMN],
                     date_format=DATE_FORMAT)
    avg_return = (df["Depotwert"].iloc[-1] / df["Depotwert"].iloc[0]) ** (
            365 / (df[DATE_COLUMN].iloc[-1] - df[DATE_COLUMN].iloc[0]).days) - 1
    df["Avg scenario"] = get_avg_return_scenario(df, avg_return)
    df["Fixed scenario"] = get_avg_return_scenario(df, FIXED_SCENARIO_RETURN)
    return df, avg_return

In [ ]:
net_worth_history, avg_return = load_net_worth_history("~/Desktop/net_worth_history.csv")
net_worth_history.head()

In [ ]:
def load_depot_proposition_history(filename: str) -> pd.DataFrame:
    """Load data from csv file and return it as a pandas dataframe"""
    df = pd.read_csv(filename, sep=";", decimal=".", parse_dates=[DATE_COLUMN],
                     date_format=DATE_FORMAT)
    return df

In [ ]:
def get_stock_quotes(depot_proposition_history: pd.DataFrame) -> pd.DataFrame:
    """Load stock quotes for the stocks in the depot proposition from yahoo."""
    positions = list(depot_proposition_history.keys()[1:])
    fin = YahooFinancials(positions, country="DE")
    sheet = fin.get_historical_price_data("2021-01-01", datetime.date.today().isoformat(), "daily")
    data = {}
    max_count = max([len(sheet[position]["prices"]) for position in positions])
    for position in positions:
        count = len(sheet[position]["prices"])
        data[position] = np.pad([sheet[position]["prices"][i]["close"] for i in
                                 range(len(sheet[position]["prices"]))], (max_count - count, 0),
                                constant_values=sheet[position]["prices"][0]["close"])
    df = pd.DataFrame()
    position = list(sheet.keys())[0]
    df[DATE_COLUMN] = [
        pd.Timestamp.fromisoformat(sheet[position]["prices"][i]["formatted_date"]) for i
        in
        range(len(sheet[position]["prices"]))]
    for position in positions:
        df[position] = data[position]
    return df


In [ ]:
def interpolate_data_nonlinear(data: pd.DataFrame, start_date: Optional[pd.Timestamp] = None,
                               end_date: Optional[pd.Timestamp] = None) -> pd.DataFrame:
    """Interpolate data using a nonlinear interpolation method."""
    keys = data.keys()[1:]
    last_values = {}
    new_data = {DATE_COLUMN: []}
    date = start_date if start_date else data[DATE_COLUMN].iloc[0]
    end_date = end_date if end_date else data[DATE_COLUMN].iloc[-1]
    for key in keys:
        last_values[key] = data[key].iloc[0]
        new_data[key] = []
    while date < end_date:
        date_idx = data[DATE_COLUMN][data[DATE_COLUMN] == date].index
        if len(date_idx) > 0:
            for key in keys:
                value = data[key].iloc[date_idx[0]]
                last_values[key] = value
                new_data[key].append(value)
        else:
            for key in keys:
                new_data[key].append(last_values[key])
        new_data[DATE_COLUMN].append(date)
        date += datetime.timedelta(days=1)
    return pd.DataFrame(new_data)

In [ ]:
def get_depot_history() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    proposition = load_depot_proposition_history("~/Desktop/depot_proposition.csv")
    quotes = get_stock_quotes(proposition)
    start_date = proposition[DATE_COLUMN].iloc[0]
    end_date = proposition[DATE_COLUMN].iloc[-1]
    proposition_history = interpolate_data_nonlinear(proposition)
    quotes_history = interpolate_data_nonlinear(quotes, start_date, end_date)
    values_history = pd.DataFrame()
    values_history[DATE_COLUMN] = proposition_history[DATE_COLUMN]
    for position in proposition.keys()[1:]:
        values_history[position] = proposition_history[position] * quotes_history[position]
    return proposition_history, quotes_history, values_history

In [ ]:
proposition_history, quote_history, value_history = get_depot_history()

# Fortune history line plot

Using just a line diagram.

In [ ]:
fig = go.Figure()
delta_depot_value = net_worth_history["Depotwert"].diff()
fig.add_trace(
    go.Scatter(x=net_worth_history[DATE_COLUMN], y=net_worth_history["Depotwert"],
               name="Depotwert"))
fig.add_trace(
    go.Scatter(x=net_worth_history[DATE_COLUMN], y=delta_depot_value, name="Depotschwankung"))
fig.add_trace(
    go.Scatter(x=net_worth_history[DATE_COLUMN], y=net_worth_history["Avg scenario"],
               name=f"Avg Scenario ({round(avg_return * 100, 2)}%)"))
fig.add_trace(go.Scatter(x=net_worth_history[DATE_COLUMN], y=net_worth_history["Fixed scenario"],
                         name=f"Fixed Scenario ({round(FIXED_SCENARIO_RETURN * 100, 2)}%)"))
fig.add_trace(
    go.Scatter(x=net_worth_history[DATE_COLUMN], y=net_worth_history["Net Worth"],
               name="Net Worth"))
fig.add_trace(go.Scatter(x=net_worth_history[DATE_COLUMN], y=net_worth_history["Davon nicht Depot"],
                         name="Davon nicht Depot"))
pass

In [ ]:
fig.show()

# Fortune proposition history

Using a stacked area chart.

In [ ]:
fig = px.area(net_worth_history, x=DATE_COLUMN, y=["Depotwert", "Davon nicht Depot"],
              title="Depotwert")
fig.add_trace(
    go.Scatter(x=net_worth_history[DATE_COLUMN], y=net_worth_history["Net Worth"], name="Net Worth",
               line=go.scatter.Line(color="limegreen", dash="dash")))
fig.show()

# Current fortune proposition (pie)

In [ ]:
labels = ["Depotwert", "Davon nicht Depot"]
display_values = [net_worth_history["Depotwert"].iloc[-1],
                  net_worth_history["Davon nicht Depot"].iloc[-1]]
fig = go.Figure(data=[go.Pie(labels=labels, values=display_values)])
fig.show()

# Depot value fluctuation histogram

In [ ]:
delta_depot_value = delta_depot_value[1:]
stddev = delta_depot_value.std()
median = delta_depot_value.median()
max_count = delta_depot_value.value_counts().max()

In [ ]:
def calculate_share_of_values_being_within_stddev(k) -> float:
    return len(
        [i for i in delta_depot_value if median - k * stddev <= i <= median + k * stddev]) / len(
        delta_depot_value)

In [ ]:
depot_value_fluctuations = sorted(delta_depot_value)
fig = px.histogram(depot_value_fluctuations, marginal="box", nbins=10)
colors = ["red", "blue", "green", "orange", "purple"]
fig.add_vline(x=median, line_width=3, line_dash="dash", line_color=colors[0])
for k in range(1, 3):
    fig.add_vline(x=median + k * stddev, line_width=3, line_dash="dash", line_color=colors[k])
    fig.add_annotation(x=median + k * stddev,
                       text=f"{round(calculate_share_of_values_being_within_stddev(k) * 100, 2)}%",
                       showarrow=False)
    fig.add_vline(x=median - k * stddev, line_width=3, line_dash="dash", line_color=colors[k])
    fig.add_annotation(x=median - k * stddev,
                       text=f"{round(calculate_share_of_values_being_within_stddev(k) * 100, 2)}%",
                       showarrow=False)

In [ ]:
fig.show()

# Net worth bubble chart

In [ ]:
sizes = net_worth_history["Depotwert"]
sizes_normalized = (sizes - sizes.min()) / (sizes.max() - sizes.min())
bubble_chart = go.Scatter(
    x=net_worth_history[DATE_COLUMN],
    y=net_worth_history["Net Worth"],
    mode='markers',
    marker=dict(
        size=sizes_normalized,
        sizemode='diameter',
        sizeref=sizes_normalized.max() / 30,
        sizemin=3,
        color=net_worth_history["Depotwert"],
        colorscale='Viridis',
        showscale=True
    ),
)
fig = go.Figure(data=[bubble_chart])
fig.update_layout(
    title='Net Worth',
    xaxis_title='Datum',
    yaxis_title='Net Worth',
)
fig.show()

# Depot proposition

In [ ]:
labels = proposition_history.keys()[1:]
values = proposition_history.iloc[-1][1:]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)],
                layout_title_text="Depot Proposition (shares)")
fig.show()

In [ ]:
labels = value_history.keys()[1:]
values = value_history.iloc[-1]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo="label+percent+value")],
                layout_title_text="Depot Proposition (value)")
fig.show()

In [ ]:
fig = px.line(quote_history, x=DATE_COLUMN, y=labels, title="Quotes")
fig.show()

In [ ]:
fig = px.line(proposition_history, x=DATE_COLUMN, y=labels, title="Stock Counts")
fig.show()

In [ ]:
fig = px.line(value_history, x=DATE_COLUMN, y=labels, title="Stock Values")
fig.show()

In [ ]:
fig = px.area(value_history, x=DATE_COLUMN, y=labels, title="Stock Values")
fig.show()